# imports

In [ ]:
!pip install dython

In [2]:
# !pip install transformers timm flash_attn einops opendatasets dython

In [4]:
import numpy as np
import pandas as pd
import gc
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import xception, ResNet50, EfficientNetV2L, VGG16
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow.keras as keras

from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy

import zipfile
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

import math
from sklearn import preprocessing
from skimage.transform import resize
from skimage import io
import albumentations as album

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import classification_report
from sklearn.utils import resample

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# classification  

In [6]:
pd.options.display.max_rows = 100

In [7]:
art_dir = '/kaggle/input/toy-art-data/toy_dataset/toy_dataset'
data_desrc_dir = '/kaggle/input/toy-art-data/toy_dataset_label.csv'

In [8]:
data_descr = pd.read_csv(data_desrc_dir, sep='\t')

In [9]:
# data_descr.sample(n=10)

In [10]:
labels_df = data_descr.drop(columns=['TITLE', 'DATE', 'LOCATION', 'URL', 'BORN-DIED'])
# labels_df.sample()

In [11]:
labels_df['TECHNIQUE'] = labels_df['TECHNIQUE'].str.extract('(([a-zA-Z]+\s*)+,?)').replace(',','', regex=True)[0]
techniques = labels_df['TECHNIQUE'].value_counts()[labels_df['TECHNIQUE'].value_counts() >= 30].index.tolist()
# techniques, len(techniques)

In [12]:
for tech_label in labels_df['TECHNIQUE'].unique():
  if tech_label not in techniques:
    labels_df['TECHNIQUE'] = labels_df['TECHNIQUE'].replace(tech_label, '-')

In [13]:
# len(labels_df['TECHNIQUE'].unique())

In [14]:
# labels_df['AUTHOR'].value_counts()

In [15]:
for label in labels_df['AUTHOR'].value_counts().index:
  if labels_df['AUTHOR'].value_counts()[label] < 10:
    labels_df['AUTHOR'] = labels_df['AUTHOR'].replace(label, '-')
  if ('MINIATURIST' or 'PAINTER' or 'UNKNOWN' or 'SCULPTOR') in label:
    labels_df['AUTHOR'] = labels_df['AUTHOR'].replace(label, 'UNKNOWN')

In [16]:
# labels_df['AUTHOR'].value_counts()

In [17]:
# labels_df['TECHNIQUE'].value_counts()

In [18]:
# labels_df['FORM'].value_counts()

In [19]:
labels_df = labels_df[(labels_df['FORM'] == 'painting') | (labels_df['FORM'] == 'tapestry') | (labels_df['FORM'] == 'graphics')] 
# labels_df.shape

In [20]:
# labels_df['TYPE'].value_counts()

In [21]:
# labels_df['SCHOOL'].value_counts()

In [22]:
# labels_df['TIMELINE'].value_counts()

In [ ]:
categorical_features=identify_nominal_columns(labels_df)
categorical_features

In [ ]:
complete_correlation = associations(labels_df.drop(columns=['FILE', 'ID']), figsize=(10,10))

In [25]:
for id in ['TECHNIQUE', 'TYPE', 'TIMELINE', 'SCHOOL']:
    for val in labels_df[id].value_counts().index:
        avg = int(labels_df[id].value_counts().mean()) + 500
        if labels_df[id].value_counts()[val] <= avg:
            minority_class = labels_df[labels_df[id] == val]
            majority_class = labels_df[labels_df[id] != val]
            minority_upsampled = resample(minority_class, replace=True, n_samples=avg, random_state=42)
            labels_df = pd.concat([majority_class, minority_upsampled])
        else: 
            majority_class = labels_df[labels_df[id] == val]
            minority_class = labels_df[labels_df[id] != val]
            majority_downsampled = resample(majority_class, replace=False, n_samples=avg, random_state=42)
            labels_df = pd.concat([minority_class, majority_downsampled])

labels_df.reset_index(inplace=True)

In [26]:
labels_df = labels_df.dropna()

In [27]:
labels_df.to_csv('labels_new.csv', index=False)

In [28]:
labels_df['TUPLES'] = [(labels_df['TECHNIQUE'][i], labels_df['TYPE'][i],labels_df['SCHOOL'][i], labels_df['TIMELINE'][i]) for i in labels_df.index]

In [29]:
binarizer = MultiLabelBinarizer()
y = binarizer.fit_transform(labels_df['TUPLES'])
# binarizer.classes_

In [ ]:
y.shape

# some functions


In [31]:
def visualize(**images):
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]);
        plt.yticks([])
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

In [32]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [33]:
def get_training_augmentation():
    train_transform = [
        album.OneOf(
            [
                album.HorizontalFlip(p=1),
                album.VerticalFlip(p=1),
                album.RandomRotate90(p=1),
            ],
            p=0.75,
        ),
    ]
    return album.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
        album.PadIfNeeded(min_height=256, min_width=256, always_apply=True),
    ]
    return album.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn=None):
    _transform = []
    if preprocessing_fn:
        _transform.append(album.Lambda(image=preprocessing_fn))
    _transform.append(album.Lambda(image=to_tensor, mask=to_tensor))

    return album.Compose(_transform)

In [34]:
def plot_graphs(history, string, model_name):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend(['train_'+string, 'val_'+string])
  plt.savefig(model_name + string + '.png')
  plt.show()

# data loaders


In [35]:
class IconClassDataloader(keras.utils.PyDataset):
    def __init__(
            self,
            images_dir,
            image_data,
            y_encoded,
            shape = (256, 256, 3),
            batch_size = 16,
            augmentation=None,
            preprocessing=None,
    ):
        self.image_data = image_data
        self.x = [os.path.join(images_dir, image_name) for image_name in image_data]
        self.y = y_encoded

        self.batch_size = batch_size
        self.shape = shape
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __len__(self):
        return int(math.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        low = int(idx * self.batch_size)
        high = int(min(low + self.batch_size, len(self.x)))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]
        x_return = []
        
        for file_name in batch_x:
            try:
                image=io.imread(file_name)
                if self.augmentation:
                    sample = self.augmentation(image=image)
                    image = sample['image']
                if self.preprocessing:
                    sample = self.preprocessing(image=image)
                    image = sample['image']
    
                x_return.append(image)
            
            except OSError:
                batch_y = np.delete(np.array(batch_y), batch_x.index(file_name), 0)

        return np.array([resize(x, self.shape) for x in x_return]), np.array(batch_y)


In [36]:
BATCH_SIZE = 64
IMAGE_SIZE = (299, 299, 3)

In [37]:
check_dataloader = IconClassDataloader(
    images_dir = art_dir,
    image_data = labels_df['FILE'].values,
    y_encoded = y,
    batch_size = 4,
    shape = IMAGE_SIZE,
    augmentation=get_training_augmentation()
)

In [ ]:
image, label = check_dataloader[random.randint(0, len(check_dataloader)-1)]
for j in range(len(image)):
    visualize(
        original_image = image[j],
    )

In [39]:
X_train, X_test, y_train, y_test = train_test_split(labels_df, labels_df['TUPLES'],
                               test_size=0.20,
                               random_state=42
                              )

In [40]:
training_dataloader = IconClassDataloader(
    images_dir = art_dir,
    image_data = X_train['FILE'].values,
    y_encoded = binarizer.transform(y_train),
    batch_size = BATCH_SIZE,
    shape = IMAGE_SIZE,
    augmentation=get_training_augmentation()
)
test_dataloader = IconClassDataloader(
    images_dir = art_dir,
    image_data = X_test['FILE'].values,
    y_encoded = binarizer.transform(y_test),
    batch_size = BATCH_SIZE,
    shape = IMAGE_SIZE,
    augmentation=get_validation_augmentation()
)

In [41]:
len(y_train), len(y_test)

(24996, 6249)

# models

In [42]:
EPOCHS = 10

In [43]:
def create_model(base_model, input_shape, output_shape, base_trainable = False, tpu = False):
    base_model.trainable = base_trainable
    if tpu:
        with tpu_strategy.scope():
            model = keras.Sequential([
                keras.Input(shape=input_shape),
                base_model,
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(256, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(output_shape, activation='sigmoid')
                ])
            model.compile(optimizer='adamW',
                          loss=tf.losses.BinaryCrossentropy(),
                          metrics=[tf.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), 
                                   tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])
    else: 
        model = keras.Sequential([
                keras.Input(shape=input_shape),
                base_model,
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(256, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(output_shape, activation='sigmoid')
                ])
        model.compile(optimizer='adamW',
                      loss=tf.losses.BinaryCrossentropy(),
                      metrics=[tf.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), 
                               tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])
    return model

In [ ]:
resnet = ResNet50(weights="imagenet", include_top=False, pooling="avg")
efficientnet = EfficientNetV2L(weights="imagenet", include_top=False, pooling="avg")
xceptionm = xception.Xception(weights="imagenet", include_top=False, pooling="avg")
vgg = VGG16(weights="imagenet", include_top=False, pooling="avg")

In [45]:
y.shape

(31245, 117)

**VGG**

In [ ]:
model_name = "model_vgg.keras"
model_vgg = create_model(vgg, IMAGE_SIZE, output_shape = y.shape[1], base_trainable=True)
model_vgg.summary()

In [47]:
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

history_vgg = model_vgg.fit(training_dataloader, epochs=EPOCHS,
                          validation_data=test_dataloader,
                          callbacks=[checkpoint,earlystopping]
                       )

In [48]:
plot_graphs(history_vgg, "loss", model_name)

In [49]:
plot_graphs(history_vgg, "precision_1", model_name)

**RESNET**

In [50]:
EPOCHS = 5

In [ ]:
model_name = "model_rn.keras"
model_rn = create_model(resnet, IMAGE_SIZE, output_shape = y.shape[1])
model_rn.summary()

In [52]:
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 1, verbose = 1, restore_best_weights=True)

history_rn = model_rn.fit(training_dataloader, epochs=EPOCHS,
                          validation_data=test_dataloader,
                          callbacks=[checkpoint,earlystopping]
                       )

In [53]:
plot_graphs(history_rn, "loss", model_name)

In [54]:
plot_graphs(history_rn, "precision_2", model_name)

**EfficientNet**

In [55]:
EPOCHS = 5

In [56]:
model_name = "model_en.keras"
model_en = create_model(efficientnet, IMAGE_SIZE, output_shape = y.shape[1], base_trainable = False)
model_en.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-l (Functional)   │ (None, 1280)           │   117,746,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 117)            │        30,069 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,569,237 (452.31 MB)

 Trainable params: 819,829 (3.13 MB)

 Non-trainable params: 117,749,408 (449.18 MB)

In [ ]:
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 1, verbose = 1, restore_best_weights=True)

history_en = model_en.fit(training_dataloader, epochs=EPOCHS,
                          validation_data=test_dataloader,
                          callbacks=[checkpoint,earlystopping]
                       )

In [ ]:
plot_graphs(history_en, "loss", model_name)

In [ ]:
plot_graphs(history_en, "precision_3", model_name)

**Xception**

In [ ]:
EPOCHS = 5

In [ ]:
model_name = "model_xc.keras"
model_xc = create_model(xceptionm, IMAGE_SIZE, output_shape = y.shape[1], base_trainable = False)
model_xc.summary()

In [ ]:
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 1, verbose = 1, restore_best_weights=True)

history_xc = model_en.fit(training_dataloader, epochs=EPOCHS,
                          validation_data=test_dataloader,
                          callbacks=[checkpoint,earlystopping]
                       )

In [ ]:
plot_graphs(history_xc, "loss", model_name)

In [ ]:
plot_graphs(history_xc, "precision_4", model_name)

# inference

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/model_xc.keras')

In [ ]:
y_test = test_dataloader[0:101].y
y_test.shape

In [ ]:
y_pred = model.predict(test_dataloader[0:101]).round()
y_pred.shape

In [ ]:
image, label = test_dataloader[random.randint(0, len(test_dataloader)-1)]
label = binarizer.inverse_transform(label)
label_pred = model.predict(image).round()
label_pred = binarizer.inverse_transform(label_pred)

plt.figure(figsize = (20 , 20))

for i in range(10):
    plt.subplot(5 , 5, i+1)
    plt.subplots_adjust(hspace = 0.3 , wspace = 0.3)
    plt.imshow(image[i])
    plt.title(f'Predicted: {label_pred}')
    plt.axis("off")

In [ ]:
print(label)

In [ ]:
print(label_pred)